### 決定木　パラメータチューニング

In [1]:
import numpy as np
import pandas as pd

from sklearn import tree
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.metrics import  mean_squared_error, make_scorer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree.export import export_text
from sklearn import preprocessing

import graphviz

C:\Users\owner\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.tree.export module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.tree. Anything that cannot be imported from sklearn.tree is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
# pickle ファイルから読み込み
train_pkl = pd.read_pickle('./train.pk3')

In [3]:
pd.__version__

'1.0.3'

In [4]:
train_pkl.shape

(55369, 13)

In [5]:
# 訓練とテストデータに分割
train, test = train_test_split(train_pkl, test_size=0.2, random_state=42)

In [6]:
# ターゲットと特徴量の分割
train_X_tmp = train.copy()
train_X_tmp.drop(['id', 'y'], axis=1, inplace=True)
train_X = train_X_tmp.iloc[:].values
train_y = train.y.values

### 検証するパラメータの設定

In [7]:
grid_param = {
    'criterion': ['mse'],
    'splitter': ['best', 'random'],
    'max_depth': [i for i in range(1, 11, 2)],
    'min_samples_split': [i for i in range(2, 11, 2)],
    'min_samples_leaf': [i for i in range(1, 11, 2)],
    'random_state': [42]
}


In [8]:
grid_param = {
    'criterion': ['mse'],
    'splitter': ['best', 'random'],
    'max_depth': [i for i in range(5, 6)],
    'min_samples_split': [i for i in range(2, 11, 2)],
    'min_samples_leaf': [i for i in range(1, 11, 2)],
    'random_state': [42]
}


In [9]:
def rmse(y, y_pred):
    ret = np.sqrt(mean_squared_error(y, y_pred))
    print("RMSE=", ret)
    return ret

### グリッドサーチ

In [10]:
gs = GridSearchCV(estimator=DecisionTreeRegressor(**grid_param), param_grid=grid_param, scoring=make_scorer(mean_squared_error, greater_is_better=False), cv=5, return_train_score=False)

In [11]:
gs.fit(train_X, train_y)

GridSearchCV(cv=5, error_score=nan,
             estimator=DecisionTreeRegressor(ccp_alpha=0.0, criterion=['mse'],
                                             max_depth=[5], max_features=None,
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=[1, 3, 5, 7, 9],
                                             min_samples_split=[2, 4, 6, 8, 10],
                                             min_weight_fraction_leaf=0.0,
                                             presort='deprecated',
                                             random_state=[42],
                                             splitter=['best', 'random']),
             iid='deprecated', n_jobs=None,
             param_grid={'criterion': ['mse'], 'max_depth': [5],
                         'min_samples_leaf': [1,

### 一番良かったパラメータで学習

In [12]:
gs.best_score_

-16925.41215404882

In [13]:
np.sqrt(abs(gs.best_score_))

130.09770233962175

In [14]:
gs.best_params_

{'criterion': 'mse',
 'max_depth': 5,
 'min_samples_leaf': 9,
 'min_samples_split': 2,
 'random_state': 42,
 'splitter': 'best'}

In [15]:
# gs.cv_results_['mean_test_score']

In [16]:
DT = DecisionTreeRegressor(**gs.best_params_)

In [17]:
DT = DT.fit(train_X, train_y)

### ツリーの可視化

In [18]:
# tree.plot_tree(DT.fit(train_X, train_y))

In [19]:
# export_text(DT, feature_names=train.iloc[:, 1:].columns)

In [20]:
# train.Survived.name

In [21]:
len(train.iloc[:, 1:].columns), train.iloc[:, 1:].columns

(12,
 Index(['accommodates', 'bathrooms', 'bedrooms', 'host_response_rate',
        'number_of_reviews', 'y', 'cancellation_policy_moderate',
        'cancellation_policy_strict', 'cancellation_policy_super_strict_30',
        'cancellation_policy_super_strict_60', 'room_type_Private room',
        'room_type_Shared room'],
       dtype='object'))

In [22]:
# 訓練済みの決定木を視覚化
dot_data = tree.export_graphviz(DT, out_file=None,
                               feature_names=train.iloc[:, 1:-1].columns,
                               class_names=train.y.name,
                               rounded=True,
                               filled=True,
                               special_characters=True)

In [23]:
# graph = graphviz.Source(dot_data)
# graph

In [24]:
# graph.write('.\DT.png')

In [25]:
type(train_X)

numpy.ndarray

In [26]:
DT.feature_importances_

array([0.03005826, 0.32233788, 0.5129272 , 0.02667334, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.08728303,
       0.02072028])

In [27]:
# 特徴量の重要度が高い順に表示
print("特徴量の重要度が高い順：")
# sorted：reverse=True 降順
print(sorted(
    zip(map(lambda x: round(x, 3), DT.feature_importances_), train.iloc[:, [1, 2, 3, 4, 5, 7, 8, 9, 10, 11]].columns),
    reverse=True))

特徴量の重要度が高い順：
[(0.513, 'bedrooms'), (0.322, 'bathrooms'), (0.087, 'room_type_Private room'), (0.03, 'accommodates'), (0.027, 'host_response_rate'), (0.0, 'number_of_reviews'), (0.0, 'cancellation_policy_super_strict_60'), (0.0, 'cancellation_policy_super_strict_30'), (0.0, 'cancellation_policy_strict'), (0.0, 'cancellation_policy_moderate')]


In [28]:
sorted(
    zip(map(lambda x: round(x, 3), DT.feature_importances_), train.iloc[:, [1, 2, 3, 4, 5, 7, 8, 9, 10, 11]].columns),
    reverse=True)

[(0.513, 'bedrooms'),
 (0.322, 'bathrooms'),
 (0.087, 'room_type_Private room'),
 (0.03, 'accommodates'),
 (0.027, 'host_response_rate'),
 (0.0, 'number_of_reviews'),
 (0.0, 'cancellation_policy_super_strict_60'),
 (0.0, 'cancellation_policy_super_strict_30'),
 (0.0, 'cancellation_policy_strict'),
 (0.0, 'cancellation_policy_moderate')]

### テストデータで予測

In [29]:
# ターゲットと特徴量の分割
test_X_tmp = test.copy()
test_X_tmp.drop(['id', 'y'], axis=1, inplace=True)
test_x = test_X_tmp.iloc[:].values
test_y = test.y.values

In [30]:
test_x.shape, test_y.shape

((11074, 11), (11074,))

In [31]:
pred_y = DT.predict(test_x)

In [32]:
np.sqrt(mean_squared_error(test_y, pred_y))

127.83799102257464

### 検証データで予測

In [33]:
# 検証データ読み込み
valid = pd.read_pickle('./test.pk3')

In [34]:
valid.shape

(18528, 12)

In [35]:
# ID の保存
valid_pass = valid.id.values

In [36]:
valid_X = valid.iloc[:, 1:].values

In [37]:
valid_X.shape, train_X.shape

((18528, 11), (44295, 11))

In [38]:
pred_valid_y = DT.predict(valid_X)

In [39]:
pred_valid_y.shape

(18528,)

In [40]:
type(valid_pass), type(pred_valid_y)

(numpy.ndarray, numpy.ndarray)

In [41]:
result_df = pd.DataFrame(pred_valid_y, valid_pass, columns=['y'])

In [42]:
result_df.to_csv("./tree_5.csv", header=False)